In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
from typing import Optional

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langsmith import traceable
from IPython.display import Image, display
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict
from typing import Literal

from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel


load_dotenv()

# Vanilla workflow

In [ ]:
class State(BaseModel):
    input: str
    type: Optional[
        Literal["write_article", "generate_table_of_contents", "review_article"]
    ] = None
    output: Optional[str] = None


class MessageType(BaseModel):
    type: Literal["write_article", "generate_table_of_contents", "review_article"]


model = ChatOpenAI(model="gpt-4.1-mini")


def classify_message(state: State) -> State:
    model_with_str_output = model.with_structured_output(MessageType)
    messages = [
        SystemMessage(
            content="You are a writer. You will classify the message into one of the following categories: 'write_article', 'generate_table_of_contents', 'review_article'."
        ),
        HumanMessage(content=f"Classify the message: {state.input}"),
    ]
    return model_with_str_output.invoke(messages).type


@traceable
def write_article(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will write an article about the topic provided."
        ),
        HumanMessage(content=f"Write an article about {state.input}"),
    ]
    return model.invoke(messages).content


@traceable
def generate_table_of_contents(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will generate a table of contents for an article about the topic provided."
        ),
        HumanMessage(
            content=f"Generate a table of contents for an article about {state.input}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def review_article(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will review the article for the topic provided."
        ),
        HumanMessage(content=f"Review the article for the topic {state.input}"),
    ]
    return model.invoke(messages).content


@traceable
def writer_chatbot(message: str) -> str:
    state = State(input=message)
    state.type = classify_message(state)
    if state.type == "write_article":
        return write_article(state)
    elif state.type == "generate_table_of_contents":
        return generate_table_of_contents(state)
    elif state.type == "review_article":
        return review_article(state)
    else:
        return "I'm sorry, I don't know how to handle that message."


writer_chatbot("Give me a table of contents for an article about Novak Djokovic")

# LangGraph implementation

## Exercise
Implement a LangGraph workflow that routes messages to the correct node.